# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
#Import libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
#read the file from the url
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'

df = pd.read_csv(url)
df

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A


In [9]:
def clean_dataframe(df):
    # 1. Check for missing values
    print("Checking for missing values...")
    print("Columns with missing values:")
    print(df.isna().sum())
    
    # 2. Convert column names to lowercase and replace spaces with underscores
    print("\nStandardizing column names...")
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # 3. Replace state abbreviations with full names
    state_mapping = {
        "AZ": "Arizona",
        "Cali": "California",
        "WA": "Washington"
    }
    if 'st' in df.columns:
        df['st'] = df['st'].map(state_mapping).fillna(df['st'])
    
    # 4. Standardize education column values
    if 'education' in df.columns:
        df['education'] = df['education'].str.replace('Bachelors', 'Bachelor', regex=False)
        df['education'] = df['education'].str.replace('College', 'Diploma', regex=False)
    
    # 5. Clean 'customer_lifetime_value' column
    if 'customer_lifetime_value' in df.columns:
        df['customer_lifetime_value'] = df['customer_lifetime_value'].replace('%', '', regex=False).astype(float)
    
    # 6. Rename 'st' column to 'state'
    if 'st' in df.columns:
        df.rename(columns={'st': 'state'}, inplace=True)
    
    # 7. Replace luxury vehicle classes with 'Luxury'
    luxury_classes = ["Sports Car", "Luxury SUV", "Luxury Car"]
    if 'vehicle_class' in df.columns:
        df['vehicle_class'] = df['vehicle_class'].replace(luxury_classes, 'Luxury')
    
    # 8. Clean 'number_of_open_complaints' column
    def extract_middle_value(complaints):
        if isinstance(complaints, str):
            parts = complaints.split('/')
            if len(parts) == 3:
                return int(parts[1])
        elif isinstance(complaints, int):
            return complaints
        return None
    
    if 'number_of_open_complaints' in df.columns:
        df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(extract_middle_value)
        df['number_of_open_complaints'] = df['number_of_open_complaints'].astype(float)
    
    # 9. Fill null values based on column data type
    print("\nFilling null values...")
    for column in df.columns:
        if df[column].dtype in ['int64', 'int32']:
            df[column].fillna(0, inplace=True)
        elif df[column].dtype in ['float64', 'float32']:
            df[column].fillna(0.0, inplace=True)
        elif df[column].dtype == 'object':
            df[column].fillna('-', inplace=True)
    
    # 10. Convert numeric values to integers where possible
    def convert_to_int(value, column_name):
        if column_name == 'customer':
            return value
        try:
            if isinstance(value, (int, float)):
                return int(value)
            elif isinstance(value, str) and value.isdigit():
                return int(value)
            elif isinstance(value, str) and value.replace('.', '', 1).isdigit():
                return int(float(value))
            else:
                return value
        except ValueError:
            return value
    
    df = df.apply(lambda col: col.map(lambda x: convert_to_int(x, col.name)))
    
    # 11. Remove duplicate rows
    print("\nRemoving duplicate rows...")
    df = df.drop_duplicates()
    
    # 12. Reset the index
    print("\nResetting the index...")
    df = df.reset_index(drop=True)
    
    print("\nData cleaning complete. Here's the cleaned DataFrame:")
    return df

# Example usage:
# Load the dataset
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'
df = pd.read_csv(url)

# Clean the dataset
cleaned_df = clean_dataframe(df)

# Display the cleaned DataFrame
print(cleaned_df)
cleaned_df

Checking for missing values...
Columns with missing values:
unnamed:_0                       0
customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
vehicle_type                     0
month                         

C:\Users\umehc\AppData\Local\Temp\ipykernel_936\2161578663.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(0, inplace=True)
C:\Users\umehc\AppData\Local\Temp\ipykernel_936\2161578663.py:60: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing


Removing duplicate rows...

Resetting the index...

Data cleaning complete. Here's the cleaned DataFrame:
       unnamed:_0 customer       state  customer_lifetime_value response  \
0               0  DK49336     Arizona                     4809       No   
1               1  KX64629  California                     2228       No   
2               2  LZ68649  Washington                    14947       No   
3               3  XL78013      Oregon                    22332      Yes   
4               4  QA50777      Oregon                     9025       No   
...           ...      ...         ...                      ...      ...   
10905       10905  FE99816      Nevada                    15563       No   
10906       10906  KX53892      Oregon                     5259       No   
10907       10907  TL39050     Arizona                    23893       No   
10908       10908  WA60547  California                    11971       No   
10909       10909  IV32877  California                   

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809,No,Basic,Diploma,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228,No,Basic,Diploma,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332,Yes,Extended,Diploma,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707,Four-Door Car,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563,No,Premium,Bachelor,2011-01-19,Unemployed,F,...,7,Personal Auto,Personal L1,Offer3,Web,1214,Luxury,Medsize,A,1
10906,10906,KX53892,Oregon,5259,No,Basic,Diploma,2011-01-06,Employed,F,...,6,Personal Auto,Personal L3,Offer2,Branch,273,Four-Door Car,Medsize,A,1
10907,10907,TL39050,Arizona,23893,No,Extended,Bachelor,2011-02-06,Employed,F,...,2,Corporate Auto,Corporate L3,Offer1,Web,381,Luxury,Medsize,A,2
10908,10908,WA60547,California,11971,No,Premium,Diploma,2011-02-13,Employed,F,...,6,Personal Auto,Personal L1,Offer1,Branch,618,SUV,Medsize,A,2


In [11]:
cleaned_df

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809,No,Basic,Diploma,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228,No,Basic,Diploma,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332,Yes,Extended,Diploma,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707,Four-Door Car,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563,No,Premium,Bachelor,2011-01-19,Unemployed,F,...,7,Personal Auto,Personal L1,Offer3,Web,1214,Luxury,Medsize,A,1
10906,10906,KX53892,Oregon,5259,No,Basic,Diploma,2011-01-06,Employed,F,...,6,Personal Auto,Personal L3,Offer2,Branch,273,Four-Door Car,Medsize,A,1
10907,10907,TL39050,Arizona,23893,No,Extended,Bachelor,2011-02-06,Employed,F,...,2,Corporate Auto,Corporate L3,Offer1,Web,381,Luxury,Medsize,A,2
10908,10908,WA60547,California,11971,No,Premium,Diploma,2011-02-13,Employed,F,...,6,Personal Auto,Personal L1,Offer1,Branch,618,SUV,Medsize,A,2


In [17]:
final_df = cleaned_df.drop('unnamed:_0', axis=1)
final_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,DK49336,Arizona,4809,No,Basic,Diploma,2011-02-18,Employed,M,48029,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,A,2
1,KX64629,California,2228,No,Basic,Diploma,2011-01-18,Unemployed,F,0,...,1,Personal Auto,Personal L3,Offer4,Call Center,744,Four-Door Car,Medsize,A,1
2,LZ68649,Washington,14947,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,2,Personal Auto,Personal L3,Offer3,Call Center,480,SUV,Medsize,A,2
3,XL78013,Oregon,22332,Yes,Extended,Diploma,2011-01-11,Employed,M,49078,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A,1
4,QA50777,Oregon,9025,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,...,7,Personal Auto,Personal L2,Offer1,Branch,707,Four-Door Car,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,Nevada,15563,No,Premium,Bachelor,2011-01-19,Unemployed,F,0,...,7,Personal Auto,Personal L1,Offer3,Web,1214,Luxury,Medsize,A,1
10906,KX53892,Oregon,5259,No,Basic,Diploma,2011-01-06,Employed,F,61146,...,6,Personal Auto,Personal L3,Offer2,Branch,273,Four-Door Car,Medsize,A,1
10907,TL39050,Arizona,23893,No,Extended,Bachelor,2011-02-06,Employed,F,39837,...,2,Corporate Auto,Corporate L3,Offer1,Web,381,Luxury,Medsize,A,2
10908,WA60547,California,11971,No,Premium,Diploma,2011-02-13,Employed,F,64195,...,6,Personal Auto,Personal L1,Offer1,Branch,618,SUV,Medsize,A,2


In [19]:
#1. Create a new DataFrame for customers with total_claim_amount > $1,000 and response "Yes"
filtered_customers = final_df[(final_df['total_claim_amount'] > 1000) & (final_df['response'] == 'Yes')]
filtered_customers

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
189,OK31456,California,11009,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,...,1,Corporate Auto,Corporate L3,Offer2,Agent,1358,Luxury,Medsize,A,1
236,YJ16163,Oregon,11009,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,...,1,Special Auto,Special L3,Offer2,Agent,1358,Luxury,Medsize,A,1
419,GW43195,Oregon,25807,Yes,Extended,Diploma,2011-02-13,Employed,F,71210,...,2,Personal Auto,Personal L2,Offer1,Branch,1027,Luxury,Small,A,2
442,IP94270,Arizona,13736,Yes,Premium,Master,2011-02-13,Disabled,F,16181,...,8,Personal Auto,Personal L2,Offer1,Web,1261,SUV,Medsize,A,2
587,FJ28407,California,5619,Yes,Premium,High School or Below,2011-01-26,Unemployed,M,0,...,1,Personal Auto,Personal L1,Offer2,Web,1027,SUV,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,FN44127,Oregon,3508,Yes,Extended,Diploma,2011-01-05,Medical Leave,M,20978,...,1,Personal Auto,Personal L2,Offer2,Branch,1176,Four-Door Car,Small,A,1
10373,XZ64172,Oregon,10963,Yes,Premium,High School or Below,2011-02-08,Employed,M,55687,...,1,Corporate Auto,Corporate L2,Offer1,Agent,1324,Luxury,Medsize,A,2
10487,IX60941,Oregon,3508,Yes,Extended,Diploma,2011-01-05,Medical Leave,M,20978,...,1,Personal Auto,Personal L3,Offer2,Branch,1176,Four-Door Car,Small,A,1
10565,QO62792,Oregon,7840,Yes,Extended,Diploma,2011-01-14,Employed,M,58414,...,1,Personal Auto,Personal L3,Offer2,Agent,1008,Four-Door Car,Medsize,A,1


In [21]:
#2. Analyze average total_claim_amount by policy type and gender for customers who responded "Yes"
average_claims = final_df[final_df['response'] == 'Yes'].groupby(['policy_type', 'gender'])['total_claim_amount'].mean().reset_index()

# Display the results
print(average_claims)

      policy_type gender  total_claim_amount
0  Corporate Auto      F          433.295858
1  Corporate Auto      M          408.123377
2   Personal Auto      F          452.498148
3   Personal Auto      M          456.576493
4    Special Auto      F          452.857143
5    Special Auto      M          429.125000


In [23]:
#3. Analyze the total number of customers who have policies in each state
customer_count_by_state = final_df['state'].value_counts().reset_index()
customer_count_by_state.columns = ['state', 'customer_count']

# Filter states with more than 500 customers
filtered_states = customer_count_by_state[customer_count_by_state['customer_count'] > 500]

# Display the results
print(filtered_states)

        state  customer_count
0  California            4183
1      Oregon            2909
2     Arizona            1937
3      Nevada             993
4  Washington             888


In [25]:
#4. Calculate max, min, and median customer lifetime value by education level and gender
lifetime_value_stats = final_df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()

# Display the results
print(lifetime_value_stats)

              education gender    max   min  median
0              Bachelor      F  73225  1904  5640.0
1              Bachelor      M  67907  1898  5548.0
2               Diploma      F  61850  1898  5623.0
3               Diploma      M  61134  1918  6005.0
4                Doctor      F  44856  2395  5332.0
5                Doctor      M  32677  2267  5577.0
6  High School or Below      F  55277  2144  6039.5
7  High School or Below      M  83325  1940  6286.5
8                Master      F  51016  2417  5729.0
9                Master      M  50568  2272  5578.5


In [ ]:
#BONUS QUESTIONS

In [29]:
#5. Convert the 'policy_start_date' to datetime to extract month
final_df['effective_to_date'] = pd.to_datetime(final_df['effective_to_date'])
final_df['month'] = final_df['effective_to_date'].dt.month_name()

# Count policies sold by state and month
policies_sold = final_df.groupby(['state', 'month']).size().unstack(fill_value=0)

# Display the table
print(policies_sold)

month       February  January
state                        
Arizona          929     1008
California      1952     2231
Nevada           442      551
Oregon          1344     1565
Washington       425      463


In [34]:
#6. Get the top 3 states with the highest number of policies sold
top_states = final_df['state'].value_counts().nlargest(3).index

# Filter the DataFrame for these states
top_states_policies = final_df[final_df['state'].isin(top_states)]

# Count policies sold by month for the top 3 states
top_states_policies_sold = top_states_policies.groupby(['state', 'month']).size().unstack(fill_value=0)

# Display the results
print(top_states_policies_sold)

month       February  January
state                        
Arizona          929     1008
California      1952     2231
Oregon          1344     1565
